# Simulate branching process epidemic model with BranchPro

The first part of the notebook includes a forward simulation of the incidence numbers for an example branching process model. The reproduction number profile is assumed to be a step function with only one jump. The results are displayed as a barplot.

The second part of the notebook focuses on the changes in the incidence numbers profiles subject to changes in the parametrization of the branching process model. This is shown via sliders.

In [1]:
# Import libraries
import branchpro
import scipy.stats
import numpy as np
import matplotlib
import plotly.graph_objects as go

## Paramterize example branching process model

In [2]:
# Build the serial interval w_s
num_timepoints = 30
ws_mean = 2.6
ws_var = 1.5**2
theta = ws_var / ws_mean
k = ws_mean / theta
 
w_dist = scipy.stats.gamma(k, scale=theta)
disc_w = w_dist.pdf(np.arange(num_timepoints))

In [3]:
# Construct BranchProModel object
initial_r = 3
serial_interval = disc_w
m = branchpro.BranchProModel(initial_r, serial_interval)

new_rs = [3, 0.5]
start_times = [0, 15]
m.set_r_profile(new_rs, start_times)

parameters = 10 # initial number of cases
times = np.arange(num_timepoints)
cases = m.simulate(parameters, times)

print(cases)

[ 10.   0.  10.   7.   9.  16.  20.  29.  46.  64.  85. 116. 180. 247.
 295. 463.  88. 161. 136. 108.  93.  68.  54.  51.  51.  36.  36.  26.
  25.  21.]


## Plot incidence numbers for the forward simulation

In [4]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=cases,
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()


## Sensitivity analysis

### 1. Varying initial cases

In [5]:
# Slider (initital cases)
fig = go.Figure()

# Add traces, one for each slider step
for parameters in np.arange(1, 100, 1):
    cases = m.simulate(parameters, times)
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=cases,
            name='Incidences'
        )
        
#         go.Scatter(
#             visible=False,
#             line=dict(color="#00CED1", width=6),
#             name="𝜈 = " + str(step),
#             x=np.arange(0, 10, 0.01),
#             y=np.sin(step * np.arange(0, 10, 0.01)))
    )

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to intial cases: " + str(np.arange(1, 100, 1)[i])}],  # layout attribute
        label = str(np.arange(1, 100, 1)[i])
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Initial Cases: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

### 2. Varying transition time of the Rt

In [6]:
new_rs = [3, 0.5]

parameters = 100 # initial number of cases

# Slider (transition time)
fig = go.Figure()

# Add traces, one for each slider step
for T in np.arange(5, 20, 1):
    start_times = [0, T]
    m.set_r_profile(new_rs, start_times)
    
    cases = m.simulate(parameters, times)
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=cases,
            name='Incidences'
        )
        
#         go.Scatter(
#             visible=False,
#             line=dict(color="#00CED1", width=6),
#             name="𝜈 = " + str(step),
#             x=np.arange(0, 10, 0.01),
#             y=np.sin(step * np.arange(0, 10, 0.01)))
    )

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to time of change: " + str(np.arange(5, 20, 1)[i])}],  # layout attribute
        label=str(np.arange(5, 20, 1)[i])
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Time of change: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

### 3. Varying the second R number value

In [7]:
start_times = [0, 15]

parameters = 100 # initial number of cases

# Slider (2nd R value)
fig = go.Figure()

# Add traces, one for each slider step
for R0_2 in np.arange(0.1, 2, 0.1):
    new_rs = [3, R0_2]
    m.set_r_profile(new_rs, start_times)
    
    cases = m.simulate(parameters, times)
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=cases,
            name='Incidences'
        )
        
#         go.Scatter(
#             visible=False,
#             line=dict(color="#00CED1", width=6),
#             name="𝜈 = " + str(step),
#             x=np.arange(0, 10, 0.01),
#             y=np.sin(step * np.arange(0, 10, 0.01)))
    )

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to second R: " + "{:.2f}".format(np.arange(0.1, 2, 0.1)[i])}],  # layout attribute
        label="{:.2f}".format(np.arange(0.1, 2, 0.1)[i])
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Second R value: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()